In [ ]:
import pandas as pd
import os 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch import  nn
from torchvision import transforms ,datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler

from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class VARAutoencodersCNN(nn.Module):
    def __init__(self,input_dim,hidden_dim,z_dim):
        super().__init__()

        #Encoder
        self.img_2hid=nn.Conv2d(input_dim,hidden_dim,3)
        self.hid_2mu=nn.Conv2d(hidden_dim,z_dim,3)
        self.hid_2sigma=nn.Conv2d(hidden_dim,z_dim,3)

        #Decoder

        self.z_2hid=nn.Conv2d(z_dim,hidden_dim,3)
        self.hid_2img=nn.Conv2d(hidden_dim,input_dim,3)


        self.Relu=nn.ReLU()
        self.sigmoid=nn.Sigmoid()
        pass
    
    def Encoder(self,x):
        hd=self.Relu(self.img_2hid(x))
        mu,sigma=self.hid_2mu(hd),self.hid_2sigma(hd)
        return(mu,sigma)

    def Decoder(self,z):

        h=self.Relu(self.z_2hid(z))
        out=self.sigmoid(self.hid_2img(h))
        return(out)

    def forward(self,Dataset):

        mu,sigma=self.Encoder(Dataset)
        epsilon=torch.rand_like(sigma)
        z_parameterized=mu+sigma*epsilon

        x_recontructed=self.Decoder(z_parameterized)

        return(x_recontructed,mu,sigma)

In [ ]:

InputDimCNN=1
HiddenDimCNN=3
OutputDimCNN=1

LR_rate=1e-3
Num_epochs=10
batch_size=512


In [ ]:
vaeLin=VARAutoencodersLinear(InputDim,HiddenDim,OutputDim).to(device)
Optimizer=Adam(vaeLin.parameters(),lr=LR_rate)
loss_fn=nn.BCELoss(reduction="sum")

In [ ]:
TrainData=datasets.CIFAR10(root="/home/yash/CNN_Data",train=True,transform=transforms.ToTensor(),download=True)
TestData=datasets.CIFAR10(root="/home/yash/CNN_Data",train=False,transform=transforms.ToTensor(),download=True) 
#Use this for the CIFAR10 Dataset

# TrainData=datasets.MNIST(root="/home/yash/CNN_Data",train=True,transform=transforms.ToTensor(),download=True)
# TestData=datasets.MNIST(root="/home/yash/CNN_Data",train=False,transform=transforms.ToTensor(),download=True) 
TrainDataSet=DataLoader(TrainData,batch_size=batch_size)
TestDataSet=DataLoader(TestData,batch_size=batch_size)